In [1]:
!pip install plotly panel

In [2]:
import pandas as pd
import numpy as np
import plotly.express as px
import panel as pn

### Read transaction data with categories

In [4]:
# Read transactions_2022_2023_categorized.csv with correct date format
df = pd.read_csv('/Users/darshan/Documents/Projects/DataScience/Personal/FinancialDashboard/Categorized_transactions.csv')
# Convert 'Date' column to datetime with dayfirst=True (since the format is DD-MM-YYYY)
df['Date'] = pd.to_datetime(df['Date'], format='%d-%m-%Y')
# Add year and month columns
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Month Name'] = df['Date'].dt.strftime("%b")
# Remove "Transaction" and "Transaction vs category" columns if they exist
if 'Transaction' in df.columns and 'Transaction vs category' in df.columns:
    df = df.drop(columns=['Transaction', 'Transaction vs category'])
df

,Date,Transaction Names,Transaction Type,Amount (INR),Category,Year,Month,Month Name
0,2023-01-02,Software Engineer,Income,75000,Salary,2023,1,Jan
1,2023-01-12,PVR Cinemas,Expense,1307,Entertainment,2023,1,Jan
2,2023-01-14,Dream Apartment,Expense,28000,Housing,2023,1,Jan
3,2023-01-17,Disney+ Hotstar,Expense,800,Entertainment,2023,1,Jan
4,2023-01-21,Corner House,Expense,2330,Food and Drinks,2023,1,Jan
...,...,...,...,...,...,...,...,...
259,2024-12-21,Easyday,Expense,3837,Grocery Shopping,2024,12,Dec
260,2024-12-21,MTR 1924,Expense,3708,Food and Drinks,2024,12,Dec
261,2024-12-23,ACT Fibernet,Expense,1500,Internet Service Provider,2024,12,Dec
262,2024-12-24,Website Design,Income,15000,Technology,2024,12,Dec


In [6]:
# For Income rows, assign Name / Description to Category
df['Category'] = np.where(df['Transaction Type'] == 'Income', df['Transaction Names'], df['Category'])

### Make pie charts - Income/ Expense breakdown

In [14]:
def make_pie_chart(df, year, label):
    # Filter the dataset for expense transactions
    sub_df = df[(df['Transaction Type'] == label) & (df['Year'] == year)]

    color_scale = px.colors.qualitative.Set2
    
    # Update the column name here to match 'Amount (INR)' (with the space)
    pie_fig = px.pie(sub_df, values='Amount (INR)', names='Category', color_discrete_sequence=color_scale)
    
    pie_fig.update_traces(textposition='inside', direction='clockwise', hole=0.3, textinfo="label+percent")

    total_expense = df[(df['Transaction Type'] == 'Expense') & (df['Year'] == year)]['Amount (INR)'].sum()
    total_income = df[(df['Transaction Type'] == 'Income') & (df['Year'] == year)]['Amount (INR)'].sum()

    if label == 'Expense':
        total_text = "₹ " + str(round(total_expense))

        # Saving rate:
        saving_rate = round((total_income - total_expense) / total_income * 100)
        saving_rate_text = ": Saving rate " + str(saving_rate) + "%"
    else:
        saving_rate_text = ""
        total_text = "₹ " + str(round(total_income))

    pie_fig.update_layout(
        uniformtext_minsize=10,
        uniformtext_mode='hide',
        title=dict(text=label + " Breakdown " + str(year) + saving_rate_text),
        # Add annotations in the center of the donut.
        annotations=[
            dict(
                text=total_text,
                x=0.5, y=0.5, font_size=12,
                showarrow=False
            )
        ]
    )

    return pie_fig

In [15]:
income_pie_fig_2023 = make_pie_chart(df, 2023, 'Income')
income_pie_fig_2023

In [16]:
expense_pie_fig_2023 = make_pie_chart(df, 2023, 'Expense')
expense_pie_fig_2023

### Make bar charts over months in a year

In [20]:
def make_monthly_bar_chart(df, year, label):
    df = df[(df['Transaction Type'] == label) & (df['Year'] == year)]
    total_by_month = (df.groupby(['Month', 'Month Name'])['Amount (INR)'].sum()
                        .to_frame()
                        .reset_index()
                        .sort_values(by='Month')  
                        .reset_index(drop=True))
    if label == "Income":
        color_scale = px.colors.sequential.YlGn
    if label == "Expense":
        color_scale = px.colors.sequential.OrRd
    
    bar_fig = px.bar(total_by_month, x='Month Name', y='Amount (INR)', text_auto='.2s', title=label+" per month", color='Amount (INR)', color_continuous_scale=color_scale)
    # bar_fig.update_traces(marker_color='lightslategrey')
    
    return bar_fig

In [22]:
income_monthly_2023 = make_monthly_bar_chart(df, 2023, 'Income')
income_monthly_2023

In [23]:
expense_monthly_2023 = make_monthly_bar_chart(df, 2023, 'Expense')
expense_monthly_2023

### Putting all charts together into tabs for 2022/2023

In [24]:
# Pie charts
income_pie_fig_2023 = make_pie_chart(df, 2023, 'Income')
expense_pie_fig_2023 = make_pie_chart(df, 2023, 'Expense')  
income_pie_fig_2024 = make_pie_chart(df, 2024, 'Income')
expense_pie_fig_2024 = make_pie_chart(df, 2024, 'Expense')

# Bar charts
income_monthly_2023 = make_monthly_bar_chart(df, 2023, 'Income')
expense_monthly_2023 = make_monthly_bar_chart(df, 2023, 'Expense')
income_monthly_2024 = make_monthly_bar_chart(df, 2024, 'Income')
expense_monthly_2024 = make_monthly_bar_chart(df, 2024, 'Expense')

# Create tabs
tabs = pn.Tabs(
                        ('2023', pn.Column(pn.Row(income_pie_fig_2023, expense_pie_fig_2023),
                                                pn.Row(income_monthly_2023, expense_monthly_2023))),
                        ('2024', pn.Column(pn.Row(income_pie_fig_2024, expense_pie_fig_2024),
                                                pn.Row(income_monthly_2024, expense_monthly_2024))
                        )
                )
tabs.show()

Launching server at http://localhost:64016


### Create dashboard

In [ ]:
# Dashboard template
template = pn.template.FastListTemplate(
    title='Personal Finance Dashboard',
    sidebar=[pn.pane.Markdown("# Income Expense analysis"), 
             pn.pane.Markdown("This dashboard provides a detailed overview of my personal income and expenses based on my bank transactions. It categorizes spending and income patterns, helping me track monthly expenditures and savings. The data is specifically generated for my lifestyle in Bangalore, my favorite place, to offer insights into my financial habits and trends."),
             pn.pane.PNG("Picture.jpg", sizing_mode="scale_both")
             ],
    main=[pn.Row(pn.Column(pn.Row(tabs)
                           )
                ),
                ],
    # accent_base_color="#88d8b0",
    header_background="#c0b9dd",
)

template.show()

Launching server at http://localhost:64029
